In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import joblib  # For saving and loading the model

# Function to preprocess the data
def preprocess_data(df):
    # Convert Date to datetime if not already
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract temporal features
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Weekday"] = df["Date"].dt.weekday
    df["Is_Weekend"] = df["Weekday"].isin([5, 6]).astype(int)
    
    # One-hot encode categorical variables
    df = pd.get_dummies(df, columns=["Weather", "Event", "Restaurant Type", "Restaurant Address"], drop_first=True)
    
    # Drop unused columns
    df = df.drop(columns=["Date", "ID", "Restaurant ID", 'Restaurant Annual Sale'])
    return df

# Load data from local files
def load_data(file_paths):
    """
    Load and combine data from a list of file paths.
    """
    dataframes = [pd.read_excel(file) for file in file_paths]
    return pd.concat(dataframes, ignore_index=True)

# File paths for the existing restaurant datasets
file_paths = ["test_data/Central.xlsx"]

# Load and preprocess the data
central_data = load_data(file_paths)
processed_data = preprocess_data(central_data)

# Define features and target
X = processed_data.drop(columns=["Number Sold"])
y = processed_data["Number Sold"]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Train the Central Model using RandomForestRegressor
central_model_rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=10)
central_model_rf.fit(X_train, y_train)

# Evaluate the Central Model
y_pred_rf = central_model_rf.predict(X_val)
mae_rf = mean_absolute_error(y_val, y_pred_rf)
rmse_rf = mean_squared_error(y_val, y_pred_rf, squared=False)

# Save the trained model to a file
model_path = "central_model_rf.pkl"
joblib.dump(central_model_rf, model_path)

print(f"Model saved to {model_path}")

ValueError: could not convert string to float: 'Beef Roll'

In [3]:
import pandas as pd
import joblib  # For loading the model

# Function to preprocess the test data (same as training data)
def preprocess_data(df):
    # Convert Date to datetime if not already
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract temporal features
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Weekday"] = df["Date"].dt.weekday
    df["Is_Weekend"] = df["Weekday"].isin([5, 6]).astype(int)
    
    # One-hot encode categorical variables
    df = pd.get_dummies(df, columns=["Weather", "Event", "Restaurant Type"], drop_first=True)
    
    # Align columns with the training data
    # Fill missing columns with 0 if they are not present in the test data
    missing_cols = [col for col in X_train.columns if col not in df.columns]
    for col in missing_cols:
        df[col] = 0
    
    # Ensure the columns match the training data
    df = df[X_train.columns]
    return df

# Load the saved model
model_path = "Model/central_model_rf.pkl"
central_model = joblib.load(model_path)
central_model.feature_names_in_


array(['Discount', 'Price', 'Restaurant Annual Sale', 'Month', 'Day',
       'Weekday', 'Is_Weekend', 'Weather_Rainy', 'Weather_Sunny',
       'Weather_Windy', 'Event_Chinese New Year',
       'Event_Dragon Boat Festival', 'Event_Lantern Festival',
       'Event_Lunar Ghost Festival', "Event_Valentine's Day",
       'Restaurant Type_Bakery', 'Restaurant Type_Bar',
       'Restaurant Type_Bubble Tea Shop', 'Restaurant Type_Buffet',
       'Restaurant Type_Caf茅', 'Restaurant Type_Cantonese Fine Dining',
       'Restaurant Type_Casual Dining', 'Restaurant Type_Cha Chaan Teng',
       'Restaurant Type_Congee and Noodle Eatery',
       'Restaurant Type_Dim Sum Restaurant', 'Restaurant Type_Fast Food',
       'Restaurant Type_Fine Dining', 'Restaurant Type_Food Truck',
       'Restaurant Type_Hot Pot Restaurant',
       'Restaurant Type_Noodle Shop',
       'Restaurant Type_Seafood Restaurant',
       'Restaurant Type_Seafood Stall', 'Restaurant Type_Steakhouse',
       'Restaurant Type_Stre

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load the saved central model
model_path = "Model/central_model_rf.pkl"  # Replace with your central model path
try:
    central_model = joblib.load(model_path)
except FileNotFoundError:
    raise FileNotFoundError("Central model file not found. Ensure the file exists at the specified path.")

# Load new user registration data
new_user_data_path = "test_data/R121_signup.xlsx"  # Replace with your new user data path
try:
    new_user_data = pd.read_excel(new_user_data_path)
    print(new_user_data)
except FileNotFoundError:
    raise FileNotFoundError("New user signup data not found. Ensure the file exists at the specified path.")

# Preprocess the new user registration data
def preprocess_data(df, reference_columns):
    """
    Preprocesses data and ensures alignment with reference_columns.
    """
    # Convert Date to datetime if not already
    df["Date"] = pd.to_datetime(df["Date"])
    y = df['Number Sold']
    # Extract temporal features
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Weekday"] = df["Date"].dt.weekday
    df["Is_Weekend"] = df["Weekday"].isin([5, 6]).astype(int)
    
    # One-hot encode categorical variables
    df = pd.get_dummies(df, columns=["Weather", "Event", "Restaurant Type"], drop_first=True)
    
    # Align columns with the central model's features
    missing_cols = [col for col in reference_columns if col not in df.columns]
    for col in missing_cols:
        df[col] = 0
    df = df[reference_columns]
    df["Number Sold"] = y
    
    return df

# Preprocess the new user data
processed_new_user_data = preprocess_data(new_user_data, central_model.feature_names_in_)
#print(processed_new_user_data.columns)
# Define features and target for the new user data
X_new_user = processed_new_user_data.drop(columns=["Number Sold"])
y_new_user = processed_new_user_data["Number Sold"]

# Train a new model initialized from the central model
new_user_model = RandomForestRegressor(
    random_state=42,
    n_estimators=central_model.n_estimators,
    max_depth=central_model.max_depth,
)
new_user_model.fit(X_new_user, y_new_user)

# Save the new user-specific model
new_user_model_path = "new_user_model_rf.pkl" 
joblib.dump(new_user_model, new_user_model_path)
print(f"New User Model saved at: {new_user_model_path}")

          Date    ID    Name  Number Sold  Discount  Price Event Weather  \
0   2024-06-01  P008  Dish 1          313      0.00     25   NaN   Sunny   
1   2024-06-01  P008  Dish 2          362      0.00     24   NaN   Sunny   
2   2024-06-01  P008  Dish 3          308      0.00     15   NaN   Sunny   
3   2024-06-01  P008  Dish 4          320      0.00     27   NaN   Sunny   
4   2024-06-02  P009  Dish 1          674      0.20     16   NaN  Cloudy   
..         ...   ...     ...          ...       ...    ...   ...     ...   
607 2024-10-30  P041  Dish 4          486      0.25     27   NaN   Sunny   
608 2024-10-31  P042  Dish 1          276      0.00     22   NaN   Rainy   
609 2024-10-31  P042  Dish 2          334      0.00     21   NaN   Rainy   
610 2024-10-31  P042  Dish 3          242      0.00     14   NaN   Rainy   
611 2024-10-31  P042  Dish 4          234      0.00     21   NaN   Rainy   

     Restaurant Type Restaurant ID Restaurant Address  Restaurant Annual Sale  
0    Bu

In [13]:
from py2neo import Graph, Node, Relationship
import pandas as pd

# 连接到Neo4j数据库
graph = Graph("bolt://localhost:7687", auth=("neo4j", "88888888"))  # 修改为你的Neo4j地址和密码

# 加载数据
file_path = 'test_data/R121_daily.xlsx'
df = pd.read_excel(file_path)

# 清理数据库以防止冲突
#graph.delete_all()

# 遍历数据并更新/插入节点和关系
for _, row in df.iterrows():
    # 创建或更新 Restaurant 节点
    restaurant = Node(
        "Restaurant",
        id=row["Restaurant ID"],
        type=row["Restaurant Type"],
        address=row["Restaurant Address"],
        annual_sale=row["Restaurant Annual Sale"],
    )
    graph.merge(restaurant, "Restaurant", "id")

    # 创建或更新 Dish 节点
    dish = Node(
        "Dish",
        id=row["ID"],
        name=row["Name"],
        price=row["Price"],
        discount=row["Discount"],
    )
    graph.merge(dish, "Dish", "id")

    # 创建或更新 Date 节点
    date = Node("Date", date=row["Date"])
    graph.merge(date, "Date", "date")

    # 创建或更新 Weather 节点
    weather = Node("Weather", condition=row["Weather"])
    graph.merge(weather, "Weather", "condition")

    # 创建或更新 Event 节点（如果有事件）
    if pd.notna(row["Event"]):
        event = Node("Event", name=row["Event"])
        graph.merge(event, "Event", "name")
    else:
        event = None

    # 创建或更新关系: Restaurant -> Dish
    rel_sold = Relationship(restaurant, "SOLD", dish, number_sold=row["Number Sold"])
    graph.merge(rel_sold)

    # 创建或更新关系: Dish -> Date
    rel_date = Relationship(dish, "SOLD_ON", date)
    graph.merge(rel_date)

    # 创建或更新关系: Date -> Weather
    rel_weather = Relationship(date, "WEATHER", weather)
    graph.merge(rel_weather)

    # 创建或更新关系: Date -> Event（如果有事件）
    if event:
        rel_event = Relationship(date, "EVENT", event)
        graph.merge(rel_event)

print("数据已成功更新到 Neo4j！")

数据已成功导入到 Neo4j！


In [5]:
from py2neo import Graph, Node, Relationship
import pandas as pd
graph = Graph("bolt://localhost:7687", auth=("neo4j", "88888888"))  # 修改为你的Neo4j地址和密码
a = Node("Bro",add = 1,aaa = 2 )
b = Node('Bro',add = 1,aaa = 2,acc = 3)

graph.merge(b, 'Bro', 'add')

In [ ]:
from py2neo import Graph, Node, Relationship

def read_files(path):
        db =  Graph("bolt://localhost:7687", auth=("neo4j", "88888888"))
        df = pd.read_excel(path)
        for _,row in df.iterrows():
            
            restaurant = Node("Restaurant",
                            ID = str(row["Restaurant ID"]),
                            type = str(row["Restaurant Type"]),
                            Address = str(row["Restaurant Address"]))
            location = Node("Location",location = str(row["Restaurant Address"]))
            dish = Node("Dish",name = str(row["Name"]),ID = str(row["ID"]))
            record = Node("Record",time = row['Date'],number = int(row["Number Sold"]),
                        discount = float(row["Discount"]),price = float(row["Price"]))
            weather = Node("Weather",weather = str(row['Weather']))
            Event =  Node("Event",event = str(row['Event']))

            db.merge(restaurant,"Restaurant","ID")
            db.merge(location,"Location","location")
            db.merge(dish,"Dish","ID")
            db.merge(record,"Record","time")
            db.merge(weather,"Weather","weather")
            db.merge(Event,"Event","event")

            db.merge(Relationship(restaurant,"located at",location),"located at","location")
            db.merge(Relationship(restaurant,"sells",dish),)
            db.merge(Relationship(dish,"sold",record))
            db.merge(Relationship(record,"at",location))
            db.merge(Relationship(record,"with",weather))
            db.merge(Relationship(record,"during",Event))

read_files("test_data/Central.xlsx")

